In [22]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import integrate
from scipy.optimize import minimize
import math
import numpy as np
from scipy.stats import norm

In [59]:
def psi_0(t, y_t, l_star_t, r, A_star, B0_star, B_star):
    """
    Compute psi_0_t value as defined in Proposition 2
    """
    # Computing A_star(2, 0; t, t+1)
    A_star_val = A_star(2, 0, t, t+1)
    
    # Computing B0_star(2, 0; t, t+1)
    B0_star_val = B0_star(2, 0, t, t+1)
    
    # Computing B_star(2, 0; t, t+1)
    B_star_val = B_star(2, 0, t, t+1)
    
    # Handle both array and scalar cases for B_star_val
    if hasattr(B_star_val, 'T'):  # If it's an array with transpose attribute
        dot_product = np.dot(B_star_val.T, l_star_t)
    else:  # If it's a scalar
        dot_product = B_star_val * l_star_t[0]
    
    # Computing psi_0_t
    psi_0_t = np.exp(-2*r + A_star_val + B0_star_val * y_t + dot_product) - 1
    
    return psi_0_t

In [60]:
def psi_1(t, z, y_t, l_star_t, A_star, B0_star, B_star, T):
    """
    Compute psi_1_t(z) value as defined in Proposition 2
    """
    # Computing A_star(z, 0; t, T)
    A_star_val = A_star(z, 0, t, T)
    
    # Computing B0_star(z, 0; t, T)
    B0_star_val = B0_star(z, 0, t, T)
    
    # Computing B_star(z, 0; t, T)
    B_star_val = B_star(z, 0, t, T)
    
    # Computing psi_1_t(z)
    # Handle both array and scalar cases for B_star_val
    if hasattr(B_star_val, 'T'):  # If it's an array with transpose attribute
        dot_product = np.dot(B_star_val.T, l_star_t)
    else:  # If it's a scalar
        dot_product = B_star_val * l_star_t[0]
        
    psi_1_val = np.exp(A_star_val + B0_star_val * y_t + dot_product)
    
    return psi_1_val

In [61]:
def psi_2(t, z, y_t, l_star_t, r, A_star, B0_star, B_star, T):
    """
    Compute psi_2_t(z) value as defined in Proposition 2
    """
    # Computing A_star(z, 0; t+1, T)
    A_star_val_1 = A_star(z, 0, t+1, T)
    
    # Computing B0_star(z, 0; t+1, T)
    B0_star_val = B0_star(z, 0, t+1, T)
    
    # Computing B_star(z, 0; t+1, T)
    B_star_val = B_star(z, 0, t+1, T)
    
    # Computing A_star(1 + z + B0_star(z, 0; t+1, T), B_star(z, 0; t+1, T); t, t+1)
    A_star_val_2 = A_star(1 + z + B0_star_val, B_star_val, t, t+1)
    
    # Computing B0_star(1 + z + B0_star(z, 0; t+1, T), B_star(z, 0; t+1, T); t, t+1)
    B0_star_val_2 = B0_star(1 + z + B0_star_val, B_star_val, t, t+1)
    
    # Computing B_star(1 + z + B0_star(z, 0; t+1, T), B_star(z, 0; t+1, T); t, t+1)
    B_star_val_2 = B_star(1 + z + B0_star_val, B_star_val, t, t+1)
    
    # Handle both array and scalar cases for B_star_val_2
    if hasattr(B_star_val_2, 'T'):  # If it's an array with transpose attribute
        dot_product = np.dot(B_star_val_2.T, l_star_t)
    else:  # If it's a scalar
        dot_product = B_star_val_2 * l_star_t[0]
    
    # Computing psi_2_t(z)
    psi_2_val = np.exp(-r + A_star_val_1 + A_star_val_2) * \
                np.exp(B0_star_val_2 * y_t) * \
                np.exp(dot_product)
    
    return psi_2_val

In [62]:
def f_check_call(z, K):
    """
    Inverse Laplace transform of a European call option payoff
    
    Parameters:
    z: complex variable for the integration
    K: strike price
    
    Returns:
    Inverse Laplace transform value
    """
    return K**(1-z) / (z * (z - 1))

In [63]:
def f_check_put(z, K):
    """
    Inverse Laplace transform of a European put option payoff
    
    Parameters:
    z: complex variable for the integration
    K: strike price
    
    Returns:
    Inverse Laplace transform value
    """
    return K**(1-z) / (z * (z - 1)) * (-1)  # The put option payoff has opposite sign

In [64]:
def integrand(z, t, y_t, l_star_t, r, A_star, B0_star, B_star, f_check, option_params, T):
    """
    Compute the integrand for equation (8)
    
    Parameters:
    z: complex variable for the integration
    t: time point
    y_t: log-return at time t
    l_star_t: scaled factor vector at time t
    r: risk-free rate
    A_star, B0_star, B_star: Functions to compute the coefficients
    f_check: Inverse Laplace transform of the option payoff
    option_params: Parameters for the option payoff (e.g., strike price)
    
    Returns:
    Integrand value
    """
    # Computing e^((z-1)*Y_t)
    exp_term = np.exp((z-1) * y_t)
    
    # Computing psi_2_t(z) - psi_1_t(z)
    psi_diff = psi_2(t, z, y_t, l_star_t, r, A_star, B0_star, B_star, T) - \
               psi_1(t, z, y_t, l_star_t, A_star, B0_star, B_star, T)
    
    # Computing f_check(z)
    f_check_val = f_check(z, **option_params)
    
    # Computing the integrand
    result = exp_term * psi_diff * f_check_val
    
    return result

In [65]:
def risk_minimizing_hedge(t, T, y_t, l_star_t, r, A_star, B0_star, B_star, f_check, option_params, R):
    """
    Compute the risk-minimizing hedging position using equation (8)
    
    Parameters:
    t: current time point
    T: option maturity
    y_t: log-return at time t
    l_star_t: scaled factor vector at time t
    r: risk-free rate
    A_star, B0_star, B_star: Functions to compute the coefficients
    f_check: Inverse Laplace transform of the option payoff
    option_params: Parameters for the option payoff (e.g., strike price)
    R: Real part of the contour in the complex plane
    
    Returns:
    Risk-minimizing hedging position
    """
    # Computing psi_0_t
    psi_0_t = psi_0(t, y_t, l_star_t, r, A_star, B0_star, B_star)
    
    # Defining the complex contour for numerical integration
    # We'll use a contour that goes vertically through R + i*y for y in [-N, N]
    N = 100  # Limit for numerical integration
    
    # Defining the integrand function for the given parameters
    def integrand_for_quad(y):
        z = complex(R, y)
        return integrand(z, t, y_t, l_star_t, r, A_star, B0_star, B_star, f_check, option_params, T).real
    
    # Computing the integral using numerical integration
    integral_result, _ = integrate.quad(integrand_for_quad, -N, N)
    
    # Computing the final result with the complex i in the denominator
    # Since we're taking the real part of the integrand, we need to account for the i in the denominator
    # When dividing by i, it's equivalent to multiplying by -i
    # This rotates the result by -90 degrees in the complex plane
    xi_t_plus_1 = np.exp(-r * (T - t)) / (2 * np.pi * psi_0_t) * (-1) * integral_result
    
    return xi_t_plus_1

In [75]:
def create_GARCH_11_functions(params):
    """
    Create A*, B0*, and B* functions for the GARCH(1,1) model based on Appendix B
    
    Parameters:
    params: Dictionary with GARCH parameters
    
    Returns:
    A*, B0*, and B* functions
    """
    # Extract parameters
    lambda_star = params.get('lambda_star', 0)  # Risk-neutral price of risk
    alpha = params.get('alpha', 0)              # GARCH parameter alpha
    beta = params.get('beta', 0)                # GARCH parameter beta
    gamma = params.get('gamma', 0)              # GARCH parameter gamma
    r = params.get('r', 0)                      # Risk-free rate
    
    # Cache for computed values
    A_cache = {}
    B_cache = {}
    
    def compute_coefficients(u, v, t, T):
        """
        Compute A* and B* coefficients for all time steps from t to T using a bottom-up approach
        """
        # Convert v to numpy array if needed
        v = np.atleast_1d(v)
        
        # Initialize arrays to store coefficients
        A_values = [0] * (T - t + 1)  # A*[t], A*[t+1], ..., A*[T]
        B_values = [0] * (T - t + 1)  # B*[t], B*[t+1], ..., B*[T]
        
        # Set terminal conditions
        A_values[T - t] = 0
        B_values[T - t] = v[0] if len(v) == 1 else v
        
        # Compute coefficients bottom-up
        for i in range(T - t - 1, -1, -1):
            # Current time step
            current_t = t + i
            
            # Compute one-step coefficient for B*
            B_val = u * lambda_star + beta * B_values[i + 1] + \
                    0.5 * (u - 2 * alpha * gamma * B_values[i + 1])**2 / (1 - 2 * alpha * B_values[i + 1])
            B_values[i] = B_val
            
            # Compute one-step coefficient for A*
            A_val = A_values[i + 1] + u * r - 0.5 * np.log(1 - 2 * alpha * B_values[i + 1]) + \
                    0.5 * (u - 2 * alpha * gamma * B_values[i + 1])**2 / (1 - 2 * alpha * B_values[i + 1])
            A_values[i] = A_val
        
        return A_values[0], np.array([B_values[0]])
    
    def A_star(u, v, t, T):
        """
        Compute A* coefficient for GARCH(1,1)
        """
        key = ('A', u, tuple(v) if isinstance(v, np.ndarray) else v, t, T)
        if key in A_cache:
            return A_cache[key]
        
        if t == T:
            return 0  # Terminal condition
        
        if t == T - 1:
            # One-step coefficient directly from Appendix B
            v_arr = np.atleast_1d(v)
            result = u * r - 0.5 * np.log(1 - 2 * alpha * v_arr[0]) + \
                     0.5 * (u - 2 * alpha * gamma * v_arr[0])**2 / (1 - 2 * alpha * v_arr[0])
        else:
            # Compute using bottom-up approach
            result, _ = compute_coefficients(u, v, t, T)
        
        A_cache[key] = result
        return result
    
    def B0_star(u, v, t, T):
        """
        Compute B0* coefficient for GARCH(1,1)
        """
        # B0* is always 0 for GARCH models
        return 0
    
    def B_star(u, v, t, T):
        """
        Compute B* coefficient for GARCH(1,1)
        """
        key = ('B', u, tuple(v) if isinstance(v, np.ndarray) else v, t, T)
        if key in B_cache:
            return B_cache[key]
        
        if t == T:
            return v  # Terminal condition
        
        if t == T - 1:
            # One-step coefficient directly from Appendix B
            v_arr = np.atleast_1d(v)
            B_val = u * lambda_star + beta * v_arr[0] + \
                    0.5 * (u - 2 * alpha * gamma * v_arr[0])**2 / (1 - 2 * alpha * v_arr[0])
            result = np.array([B_val])
        else:
            # Compute using bottom-up approach
            _, result = compute_coefficients(u, v, t, T)
        
        B_cache[key] = result
        return result
    
    return A_star, B0_star, B_star

In [76]:
# Example usage
# Define parameters for the GARCH model
# Example usage
# Define GARCH parameters (using typical values for S&P 500)
garch_params = {
    'lambda_star': 2.0,     # Risk-neutral price of risk
    'alpha': 4.0e-6,        # GARCH parameter alpha
    'beta': 0.85,           # GARCH parameter beta
    'gamma': 150.0,         # GARCH parameter gamma (leverage effect)
    'r': 0.02               # Risk-free rate
}

# Create A*, B0*, and B* functions
A_star, B0_star, B_star = create_GARCH_11_functions(garch_params)

# Define option parameters
option_params = {
    'K': 100  # Strike price
}

# Current state
t = 0
T = 1
y_t = np.log(100)  # Log of current price
l_star_t = np.array([0.01])  # Current value of the scaled factor (e.g., conditional variance)
r = 0.02  # Risk-free rate
R = 1.5  # Real part of the contour for European call option (R > 1)

# Compute the risk-minimizing hedging position
xi_t_plus_1 = risk_minimizing_hedge(
    t, T, y_t, l_star_t, r,
    A_star, B0_star, B_star,
    f_check_call, option_params, R
)

print(f"Risk-minimizing hedging position: {xi_t_plus_1}")

Risk-minimizing hedging position: -0.7802216077943046


In [77]:
def option_price(t, T, y_t, l_star_t, r, A_star, B0_star, B_star, f_check, option_params, R):
    """
    Compute the option price using equation (9)
    
    Parameters:
    t: current time point
    T: option maturity
    y_t: log of asset price at time t
    l_star_t: scaled factor vector at time t
    r: risk-free rate
    A_star, B0_star, B_star: Functions to compute the coefficients
    f_check: Inverse Laplace transform of the option payoff
    option_params: Parameters for the option payoff (e.g., strike price)
    R: Real part of the contour in the complex plane
    
    Returns:
    Option price
    """
    # Defining the complex contour for numerical integration
    # We'll use a contour that goes vertically through R + i*y for y in [-N, N]
    N = 100  # Limit for numerical integration
    
    # Defining the integrand function for the given parameters
    def price_integrand(y):
        z = complex(R, y)
        # Computing e^{zY_t}
        exp_term = np.exp(z * y_t)
        
        # Computing ψ^{(1)}_t(z)
        psi_1_val = psi_1(t, z, y_t, l_star_t, A_star, B0_star, B_star, T)
        
        # Computing \check{f}(z)
        f_check_val = f_check(z, **option_params)
        
        # Computing the integrand
        result = exp_term * psi_1_val * f_check_val
        
        return result.real
    
    # Computing the integral using numerical integration
    integral_result, _ = integrate.quad(price_integrand, -N, N)
    
    # Computing the final result
    price = np.exp(-r * (T - t)) * integral_result / (2 * np.pi)
    
    return price

In [78]:
# Compute the option price
option_price_value = option_price(
    t, T, y_t, l_star_t, r,
    A_star, B0_star, B_star,
    f_check_call, option_params, R
)

print(f"Option price: {option_price_value}")

Option price: 93.45097988932771


In [79]:
def black_scholes_price(S, K, T, r, sigma, option_type='call'):
    """
    Calculate the price of European options using the Black-Scholes formula
    
    Parameters:
    S: Current stock price
    K: Strike price
    T: Time to maturity (in years)
    r: Risk-free interest rate (annual)
    sigma: Volatility (annual)
    option_type: 'call' or 'put'
    
    Returns:
    Option price
    """
    # Calculate d1 and d2
    d1 = (np.log(S/K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    # Calculate option price
    if option_type.lower() == 'call':
        option_price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    elif option_type.lower() == 'put':
        option_price = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    else:
        raise ValueError("option_type must be 'call' or 'put'")
    
    return option_price

In [80]:
def validate_option_pricing_model(S, K, T, r, sigma, option_type='call'):
    """
    Validate the affine option pricing model by comparing to Black-Scholes in a limit case
    
    Parameters:
    S: Current stock price
    K: Strike price
    T: Time to maturity (in years)
    r: Risk-free interest rate (annual)
    sigma: Volatility (annual)
    option_type: 'call' or 'put'
    
    Returns:
    Dictionary with validation results
    """
    # Calculate Black-Scholes price
    bs_price = black_scholes_price(S, K, T, r, sigma, option_type)
    
    # Set up GARCH parameters for a limit case (close to constant volatility)
    # For GARCH(1,1), we set:
    # - High persistence (β close to 1)
    # - Low volatility of volatility (α close to 0)
    # - No leverage effect (γ = 0)
    garch_params = {
        'omega': sigma**2 * (1 - 0.999),  # Sets the long-run variance to sigma^2
        'alpha': 0.0001,  # Very small alpha
        'beta': 0.999,    # Very high persistence
        'gamma': 0,       # No leverage effect
    }
    
    # Current state
    t = 0
    y_t = np.log(S)  # Log of current price
    # In this limit case, the conditional variance equals the long-run variance
    l_star_t = np.array([sigma**2])
    
    # Real part of the contour for integration
    R = 1.5 if option_type.lower() == 'call' else -0.5
    
    # Option parameters
    option_params = {'K': K}
    
    # Choose the appropriate payoff function
    f_check = f_check_call if option_type.lower() == 'call' else f_check_put
    
    # Define simple A_star, B0_star, and B_star functions for the limit case
    # These should mimic constant volatility
    def A_star_limit(u, v, t, T_val):
        # In a constant volatility model, this would be a simple function
        # For example, in Black-Scholes, A_star would be related to r and sigma
        if T_val is None or t == T_val:
            return 0
        else:
            # This is a simplified placeholder
            time_to_mat = T_val - t if T_val is not None else T - t
            return (r - 0.5 * sigma**2) * u * time_to_mat + 0.5 * sigma**2 * u**2 * time_to_mat
    
    def B0_star_limit(u, v, t, T_val):
        # In this simplified model, there's no autoregressive component in returns
        return 0
    
    def B_star_limit(u, v, t, T_val):
        # In a constant volatility model, the variance state doesn't affect future values
        if T_val is None or t == T_val:
            if isinstance(v, (int, float)):
                return np.array([v])
            return v
        else:
            return np.array([0.0])
    
    # Compute the option price using your model
    garch_price = option_price(
        t, T, y_t, l_star_t, r,
        lambda u, v, t, T_val: A_star_limit(u, v, t, T_val),
        lambda u, v, t, T_val: B0_star_limit(u, v, t, T_val),
        lambda u, v, t, T_val: B_star_limit(u, v, t, T_val),
        f_check, option_params, R
    )
    
    # Compare results
    abs_diff = abs(bs_price - garch_price)
    rel_diff = abs_diff / bs_price if bs_price != 0 else float('inf')
    
    return {
        'Black-Scholes Price': bs_price,
        'GARCH Model Price': garch_price,
        'Absolute Difference': abs_diff,
        'Relative Difference': rel_diff,
        'Is Valid': rel_diff < 0.05  # Consider valid if within 5%
    }

In [84]:
# Test validation
validation_result = validate_option_pricing_model(
    S=100,       # Current stock price
    K=100,       # Strike price (at-the-money)
    T=1,         # One year to maturity
    r=0.02,      # 2% risk-free rate
    sigma=0.2,   # 20% volatility
    option_type='call'
)

print("Validation Results:")
for key, value in validation_result.items():
    print(f"{key}: {value}")

Validation Results:
Black-Scholes Price: 8.916037278572539
GARCH Model Price: 8.916037278572734
Absolute Difference: 1.9539925233402755e-13
Relative Difference: 2.1915481758204476e-14
Is Valid: True


In [87]:
# Test with more moderate parameters
test_garch_params = {
    'lambda_star': 0.5,      # More moderate risk-neutral price of risk
    'alpha': 1.0e-5,         # Typical GARCH alpha
    'beta': 0.9,             # Typical GARCH beta
    'gamma': 50.0,           # More moderate leverage parameter
    'r': 0.02                # Risk-free rate
}

# Create A*, B0*, and B* functions
A_star, B0_star, B_star = create_GARCH_11_functions(garch_params)

# Current state
t = 0
T = 1  
S_t = 100
y_t = np.log(S_t)
h_t = 0.04  # Initial conditional variance (annualized)
l_star_t = np.array([h_t])  # Scaled factor vector
r = 0.02
K = 100  # Strike price (at-the-money)
option_params = {'K': K}

# Calculate option price
option_price_value = option_price(
    t, T, y_t, l_star_t, r,
    A_star, B0_star, B_star,
    f_check_call, option_params, 1.5
)

print(f"Option price: {option_price_value}")

# Calculate hedging position
xi_t_plus_1 = risk_minimizing_hedge(
    t, T, y_t, l_star_t, r,
    A_star, B0_star, B_star,
    f_check_call, option_params, 1.5
)

print(f"Risk-minimizing hedging position: {xi_t_plus_1}")

Option price: 105.35570350667574
Risk-minimizing hedging position: -0.7906647493258716
